In [1]:
%serialconnect

Connecting to Serial /dev/ttyUSB2 baud=115200 
Ready.



In [3]:
%sendtofile --source i2cmodule.py

Sent 2 lines (72 bytes).


In [5]:
import os
print(os.listdir())

['boot.py', 'aaa', 'bbb.txt', 'ddd', 'dd', 'hhh.txt', 'i2cmodule.py']


In [4]:
from i2cmodule import i2c


In [10]:
print(0x77, (i2c.scan()))

119 [119]


In [9]:
import time
i2c.writeto(0x77, b'\x1E')
time.sleep_ms(100)
C1 = readfromMS5611(0xA2, 2)
print(C1)
C2 = readfromMS5611(0xA4, 2)
print(C2)
C3 = readfromMS5611(0xA6, 2)
print(C3)
C4 = readfromMS5611(0xA8, 2)
print(C4)
C5 = readfromMS5611(0xAA, 2)
print(C5)

Traceback (most recent call last):
  File "<stdin>", line 4, in <module>
NameError: name 'readfromMS5611' is not defined


In [17]:
import ustruct

C3, C4, C6, SENST1, OFFT1, Tref = 0,0,0,0,0,0
def setupms5611():
    global C3, C4, C6, SENST1, OFFT1, Tref
    i2c.writeto(0x77, b'\x1E')
    time.sleep_ms(20)
    C1 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA2, 2))[0]
    C2 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA4, 2))[0]
    C3 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA6, 2))[0]
    C4 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA8, 2))[0]
    C5 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xAA, 2))[0]
    C6 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xAC, 2))[0]
    SENST1 =  C1*0x8000
    OFFT1 = C2*0x10000
    Tref = C5*0x100
    
setupms5611()
print(C3, C4, C6, SENST1, OFFT1, Tref)

29313 26259 28282 1556152320 3160997888 8063232


In [18]:
def MS5611readD():
    i2c.writeto(0x77, b'\x48')
    time.sleep_ms(20)
    r = i2c.readfrom_mem(0x77, 0x00, 3)
    D1 = r[0]*0x10000 + r[1]*0x100 + r[2]
    
    i2c.writeto(0x77, b'\x58')
    time.sleep_ms(20)
    r = i2c.readfrom_mem(0x77, 0x00, 3)
    D2 = r[0]*0x10000 + r[1]*0x100 + r[2]
    return D1, D2

def MS5611convert(D1, D2):
    dT = D2 - Tref
    TEMP = 2000 + dT*C6/0x00800000 
    OFF = OFFT1 + dT*C4/0x80 
    SENS = SENST1 + dT*C3 / 0x100
    if TEMP < 2000:
        T2 = dT*dT/0x80000000; 
        ra = TEMP - 2000; 
        ra = ra*ra; 
        OFF -= 5*ra/2; 
        SENS -= 5*ra / 4; 
        if TEMP < -1500:
            rb = TEMP - (-1500) 
            rb = rb*rb 
            OFF -= 7*rb 
            SENS -= 11*rb / 2 
        TEMP -= T2; 
    return (SENS * D1 / 0x200000 - OFF) / 0x8000 
 

In [19]:
print(MS5611readD())

(8753302, 8045304)


In [20]:
D1, D2 = MS5611readD()
print(MS5611convert(D1, D2))



101600.1


In [21]:
%%writefile ms5611_funcs.py

from i2cmodule import i2c

import ustruct, time

C3, C4, C6, SENST1, OFFT1, Tref = 0,0,0,0,0,0
def setupms5611():
    global C3, C4, C6, SENST1, OFFT1, Tref
    i2c.writeto(0x77, b'\x1E')
    time.sleep_ms(20)
    C1 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA2, 2))[0]
    C2 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA4, 2))[0]
    C3 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA6, 2))[0]
    C4 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA8, 2))[0]
    C5 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xAA, 2))[0]
    C6 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xAC, 2))[0]
    SENST1 =  C1*0x8000
    OFFT1 = C2*0x10000
    Tref = C5*0x100
    
def MS5611readD():
    i2c.writeto(0x77, b'\x48')
    time.sleep_ms(20)
    r = i2c.readfrom_mem(0x77, 0x00, 3)
    D1 = r[0]*0x10000 + r[1]*0x100 + r[2]
    
    i2c.writeto(0x77, b'\x58')
    time.sleep_ms(20)
    r = i2c.readfrom_mem(0x77, 0x00, 3)
    D2 = r[0]*0x10000 + r[1]*0x100 + r[2]
    return D1, D2

def MS5611convert(D1, D2):
    dT = D2 - Tref
    TEMP = 2000 + dT*C6/0x00800000 
    OFF = OFFT1 + dT*C4/0x80 
    SENS = SENST1 + dT*C3 / 0x100
    if TEMP < 2000:
        T2 = dT*dT/0x80000000; 
        ra = TEMP - 2000; 
        ra = ra*ra; 
        OFF -= 5*ra/2; 
        SENS -= 5*ra / 4; 
        if TEMP < -1500:
            rb = TEMP - (-1500) 
            rb = rb*rb 
            OFF -= 7*rb 
            SENS -= 11*rb / 2 
        TEMP -= T2; 
    return (SENS * D1 / 0x200000 - OFF) / 0x8000 


Writing ms5611_funcs.py



In [22]:
%sendtofile --source ms5611_funcs.py

Sent 49 lines (1497 bytes).


In [24]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.2-443-g236297f4 on 2017-11-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nOSError: [Errno 2] ENOENT\r\nMicroPython v1.9.2-443-g236297f4 on 2017-11-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [25]:
import ms5611_funcs

In [26]:
ms5611_funcs.setupms5611()

In [31]:
D1, D2 = ms5611_funcs.MS5611readD()
print(ms5611_funcs.MS5611convert(D1, D2))



101606.1
